In [6]:
from tqdm import tqdm
import pandas as pd
import pathlib
import json
import requests
import os

In [115]:
os.system('git submodule update --recursive --remote')

From github.com:uva-bi-sdad/sdc.broadband.acs
   7101776..93e6dc8  main       -> origin/main


Submodule path '../sdc.broadband.acs': checked out '93e6dc855e90bd06d4740e531d2bf68de55e5de2'


0

In [116]:
acs_data_dir = '../sdc.broadband.acs/data/distribution'
os.path.isdir(acs_data_dir)

True

In [11]:
ookla_data_dir = '../sdc.broadband.ookla/data/distribution'
os.path.isdir(ookla_data_dir)

True

In [33]:
bbn_data_dir = '../sdc.broadband.broadbandnow/data/distribution'
os.path.isdir(bbn_data_dir)

True

### To get percentage of household income for the price, take the price for that block group, and divide it by the median household income for that block group, and return a percentage

In [48]:
bbn_df = pd.read_csv(sorted(pathlib.Path(bbn_data_dir).glob('*.csv.xz'))[0], dtype={'GEOID20':object})
bbn_df

,speed,down_up,price,name,type,address,year_parsed,GEOID20,latitude,longitude
0,115 Mbps,Download,50.00,T-Mobile 5G Home Internet,Home Internet w/ Autopay,"721, Shows Drive, Bibb County, Alabama, 35034,...",2023,010070100112,32.893463,-87.270959
1,23 Mbps,Upload,50.00,T-Mobile 5G Home Internet,Home Internet w/ Autopay,"721, Shows Drive, Bibb County, Alabama, 35034,...",2023,010070100112,32.893463,-87.270959
2,115 Mbps,Download,55.00,T-Mobile 5G Home Internet,Home Internet,"721, Shows Drive, Bibb County, Alabama, 35034,...",2023,010070100112,32.893463,-87.270959
3,23 Mbps,Upload,55.00,T-Mobile 5G Home Internet,Home Internet,"721, Shows Drive, Bibb County, Alabama, 35034,...",2023,010070100112,32.893463,-87.270959
4,100 Mbps,Download,149.99,Viasat Internet,Unlimited Platinum 100,"721, Shows Drive, Bibb County, Alabama, 35034,...",2023,010070100112,32.893463,-87.270959
...,...,...,...,...,...,...,...,...,...,...
497,880 Mbps,Upload,164.99,Verizon Fios,Fios Gigabit Connection + Fios TV Test Drive,"Cedar Crest Road, Bibb County, Alabama, United...",2023,010070100011,33.094946,-87.258547
498,200 Mbps,Download,30.00,CenturyLink Fiber Gigabit,NaN,"Cedar Crest Road, Bibb County, Alabama, United...",2023,010070100011,33.094946,-87.258547
499,200 Mbps,Upload,30.00,CenturyLink Fiber Gigabit,NaN,"Cedar Crest Road, Bibb County, Alabama, United...",2023,010070100011,33.094946,-87.258547
500,182 Mbps,Download,50.00,T-Mobile 5G Home Internet,Home Internet w/ Autopay,"Cedar Crest Road, Bibb County, Alabama, United...",2023,010070100011,33.094946,-87.258547


In [49]:
bbn_df['speed_val'] = bbn_df['speed'].apply(lambda x: int(x.split('Mbps')[0]))

In [66]:
len(bbn_df['GEOID20'].unique())

18

In [85]:
min_up_100 = bbn_df[(bbn_df['down_up']=='Upload') & (bbn_df['speed_val'] >= 100)]
print(len(min_up_100['GEOID20'].unique()))
min_df = min_up_100.groupby(['GEOID20'])['price'].min()

17


In [78]:
# # To test that the values of the operations return the same results
# dfs = []
# for geoid in min_up_100['GEOID20'].unique():
#     pdf = min_up_100[min_up_100['GEOID20']==geoid]
#     pdf = pdf[pdf['price'] == pdf['price'].min()]
#     dfs.append(pdf)

# test_df = pd.concat(dfs)
# print(test_df)
# print(len(test_df['GEOID20'].unique()))

In [86]:
min_df = min_df.reset_index()
min_df

,GEOID20,price
0,010070100011,30.0
1,010070100012,30.0
2,010070100051,30.0
3,010070100052,30.0
4,010070100053,30.0
5,010070100061,30.0
6,010070100062,30.0
7,010070100071,30.0
8,010070100072,30.0
9,010070100081,30.0


In [117]:
income_df = pd.read_csv(os.path.join(acs_data_dir, '01007.csv.xz'), dtype={'GEOID21':object})
income_df

,B19013_001E,B19013A_001M,B19013A_001MA,GEOID21
0,45000.0,NaN,NaN,010070100011
1,34009.0,NaN,NaN,010070100012
2,58558.0,NaN,NaN,010070100051
3,36222.0,NaN,NaN,010070100052
4,80855.0,NaN,NaN,010070100053
5,83242.0,NaN,NaN,010070100061
6,67844.0,NaN,NaN,010070100062
7,56139.0,NaN,NaN,010070100071
8,59688.0,NaN,NaN,010070100072
9,-666666666.0,NaN,NaN,010070100081


In [118]:
income_df = income_df[income_df['B19013_001E']> 0] # Filter out locations where there are no data

In [119]:
income_df = income_df.rename(columns={'GEOID21': 'geoid'})
min_df = min_df.rename(columns={'GEOID20': 'geoid'})

In [120]:
min_df['geoid'].map(len).value_counts()

geoid
12    17
Name: count, dtype: int64

In [121]:
income_df['geoid'].map(len).value_counts()

geoid
12    14
Name: count, dtype: int64

In [122]:
income_df

,B19013_001E,B19013A_001M,B19013A_001MA,geoid
0,45000.0,NaN,NaN,010070100011
1,34009.0,NaN,NaN,010070100012
2,58558.0,NaN,NaN,010070100051
3,36222.0,NaN,NaN,010070100052
4,80855.0,NaN,NaN,010070100053
5,83242.0,NaN,NaN,010070100061
6,67844.0,NaN,NaN,010070100062
7,56139.0,NaN,NaN,010070100071
8,59688.0,NaN,NaN,010070100072
10,73550.0,NaN,NaN,010070100082


In [123]:
min_df

,geoid,price
0,010070100011,30.0
1,010070100012,30.0
2,010070100051,30.0
3,010070100052,30.0
4,010070100053,30.0
5,010070100061,30.0
6,010070100062,30.0
7,010070100071,30.0
8,010070100072,30.0
9,010070100081,30.0


In [124]:
merged_df = income_df.merge(min_df, how='inner', on='geoid')
merged_df

,B19013_001E,B19013A_001M,B19013A_001MA,geoid,price
0,45000.0,NaN,NaN,010070100011,30.0
1,34009.0,NaN,NaN,010070100012,30.0
2,58558.0,NaN,NaN,010070100051,30.0
3,36222.0,NaN,NaN,010070100052,30.0
4,80855.0,NaN,NaN,010070100053,30.0
5,83242.0,NaN,NaN,010070100061,30.0
6,67844.0,NaN,NaN,010070100062,30.0
7,56139.0,NaN,NaN,010070100071,30.0
8,59688.0,NaN,NaN,010070100072,30.0
9,73550.0,NaN,NaN,010070100082,30.0


In [125]:
merged_df['perc_income_min_price_100'] = merged_df['price']/merged_df['B19013_001E']*100
merged_df

,B19013_001E,B19013A_001M,B19013A_001MA,geoid,price,perc_income_min_price_100
0,45000.0,NaN,NaN,010070100011,30.0,0.066667
1,34009.0,NaN,NaN,010070100012,30.0,0.088212
2,58558.0,NaN,NaN,010070100051,30.0,0.051231
3,36222.0,NaN,NaN,010070100052,30.0,0.082823
4,80855.0,NaN,NaN,010070100053,30.0,0.037103
5,83242.0,NaN,NaN,010070100061,30.0,0.036039
6,67844.0,NaN,NaN,010070100062,30.0,0.044219
7,56139.0,NaN,NaN,010070100071,30.0,0.053439
8,59688.0,NaN,NaN,010070100072,30.0,0.050261
9,73550.0,NaN,NaN,010070100082,30.0,0.040789


In [126]:
cols = requests.get('https://raw.githubusercontent.com/uva-bi-sdad/sdc.metadata/master/data/column_structure.json').json()
final_df = pd.DataFrame(columns = cols)
final_df

,geoid,measure,moe,value,year
